In [1]:
#import packages

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

Functions

In [2]:
# Add missing ID & select interested columns individually
# type = true for i and s datasets; false for o dataset

def addID(df, type):
    
    if (type):
        df = df[['EFAMID','ENROLID','DISDATE']]
        df['DISDATE'] = pd.to_datetime(df['DISDATE'])
        df['DISDATE'] = df['DISDATE'] - timedelta(days=1)
    if(not type):
        df = df[['EFAMID','ENROLID','SVCDATE']]
        df = df.rename(columns={'EFAMID': 'EFAMID', 'ENROLID':'ENROLID', 'SVCDATE':'DISDATE'})
        df['DISDATE'] = pd.to_datetime(df['DISDATE'])
    
    for index in df.index:
        if np.isnan(df['ENROLID'][index]):
            famId = df['EFAMID'][index]
            newID = str(famId) + "_fam"
            df['ENROLID'][index] = newID
            
    df['ENROLID'].astype(str)
    df = df[['ENROLID', 'DISDATE']]
    
    return df

In [3]:
def CombineYear(df1, df2, df3):
    df_all = pd.concat([df1, df2, df3])
    df_all.reset_index(drop=False, inplace=True)
    df_all = df_all[['ENROLID', 'DISDATE']]
    return df_all.drop_duplicates()

In [4]:
# find if any previous AF occurance in the current year
def AFCodingCurrent(allData, afaData):
    allData[allData.duplicated(subset=['ENROLID'], keep='last')]
    df = pd.DataFrame(columns=[['ENROLID', 'DISDATE']])
    allData['DISDATE'] = pd.to_datetime(allData['DISDATE'])
    afaData['DISDATE'] = pd.to_datetime(afaData['DISDATE'])
    for index in afaData.index:
        if afaData['ENROLID'][index] in allData.ENROLID.values.tolist():
            df.loc[len(df.index)] = [afaData['ENROLID'][index], afaData['DISDATE'][index]] 
    return df

In [5]:
# find if any previous AF occurance in the previous year
def AFCodingPrev(allDataPrev, afaData):
    df = pd.DataFrame(columns=[['ENROLID', 'DISDATE']])
    for index in afaData.index:
        if afaData['ENROLID'][index] in allDataPrev.ENROLID.values.tolist():
            df.loc[len(df.index)] = [afaData['ENROLID'][index], afaData['DISDATE'][index]] 
    return df

In [6]:
def CombineAFCoding(df1, df2):
    df_all = pd.concat([df1, df2])
    df_all.reset_index(drop=True, inplace=True)
    return df_all.drop_duplicates()

In [8]:
# find if any reoccurence of AF within 180 days of the current year
def FindFailureCurr6(allData, afaData):
    allData = allData[allData.duplicated(subset=['ENROLID'], keep=False)]
    allData = allData.sort_values('DISDATE').drop_duplicates(['ENROLID'], keep='last')
    df = pd.DataFrame(columns=[['ENROLID', 'DISDATE']])
    for index in afaData.index:
        id_num = afaData['ENROLID'].iloc[index][0]
        indexDate = allData.index[allData.ENROLID == id_num].tolist()
        if len(indexDate) == 0:
            continue
        else:
            indexDate = indexDate[0]
            Alldate = allData['DISDATE'][indexDate] #date of last AF
            curDate = afaData['DISDATE'].iloc[index][0] #date of AFA
            if ((curDate + timedelta(days=180)) < Alldate):
                df.loc[len(df.index)] = [afaData['ENROLID'].iloc[index][0], afaData['DISDATE'].iloc[index][0]] 
    return df

In [10]:
# find if any reoccurence of AF within 180 days of the next year
def FindFailureNext6(allData, afaData):
    allData = allData[allData.duplicated(subset=['ENROLID'], keep=False)]
    allData = allData.sort_values('DISDATE').drop_duplicates(['ENROLID'], keep='last')
    df = pd.DataFrame(columns=[['ENROLID', 'DISDATE']])
    for index in afaData.index:
        id_num = afaData['ENROLID'].iloc[index][0]
        indexDate = allData.index[allData.ENROLID == id_num].tolist()
        if len(indexDate) == 0:
            continue
        else:
            indexDate = indexDate[0]
            Alldate = allData["DISDATE"][indexDate]
            curDate = afaData['DISDATE'].iloc[index][0]
            if (((curDate + timedelta(days=365)) >= Alldate) and (curDate + timedelta(days=180)) < Alldate):
                df.loc[len(df.index)] = [afaData['ENROLID'].iloc[index][0], afaData['DISDATE'].iloc[index][0]] 
    return df

In [11]:
def CombineSuccess(failure, alldata):
    failure.columns = [["ENROLID","DISDATE"]]
    alldata.columns = [["ENROLID","DISDATE"]]
    df_all = pd.concat([failure, alldata])
    df_all.reset_index(drop=False, inplace=True)
    df_all = df_all[['ENROLID', 'DISDATE']]
    return df_all.drop_duplicates(keep=False)